## Image augmentation using Albumentations.

This program uses seed function to ensure the same transformation is applied in every run, but it does not apply the transformation to the same extant (the degree of of rotations, the level of blur might vary).

In [1]:
# Importing necessary libraries

import os
import cv2
import numpy as np
import random
import albumentations as A

In [2]:
# Creating a list of seed values for applying the transformation and the number of images for each class.

seed_list = list()
sample_list = list()

random.seed(99)

for _ in range(43):
    seed_list.append(random.randint(100, 999))
    sample_list.append(random.randint(9750, 10250))

In [3]:
# Ensure you get the same values in output section as the comment below for the list of seed values.

# Array [513, 304, 283, 354, 878, 357, 492, 800, 651, 738, 304, 839, 702, 911, 907, 942, 982, 317, 572, 
# 283, 680, 184, 833, 793, 189, 423, 143, 671, 535, 471, 647, 573, 302, 573, 152, 232, 832, 865, 552, 281, 
# 615, 280, 212]
# Sum 23167
# Mean 538.7674418604652
# Min 143
# Max 982

print("Array", seed_list)
print("Sum", np.sum(np.asarray(seed_list)))
print("Mean", np.mean(np.asarray(seed_list)))
print("Min", np.min(np.asarray(seed_list)))
print("Max", np.max(np.asarray(seed_list)))

Array [513, 304, 283, 354, 878, 357, 492, 800, 651, 738, 304, 839, 702, 911, 907, 942, 982, 317, 572, 283, 680, 184, 833, 793, 189, 423, 143, 671, 535, 471, 647, 573, 302, 573, 152, 232, 832, 865, 552, 281, 615, 280, 212]
Sum 23167
Mean 538.7674418604652
Min 143
Max 982


In [4]:
# Ensure you get the same values in output section as the comment below for the number of images in each class.

# Array [9944, 10056, 9867, 9818, 9794, 10123, 10021, 10108, 9795, 10000, 9965, 10063, 9860, 10211, 9941, 9948, 
# 10099, 9827, 10088, 10185, 9923, 10234, 10020, 10146, 9990, 9956, 9887, 9753, 9854, 10227, 9791, 10015, 10060, 
# 10250, 9933, 9860, 9832, 10155, 10054, 10233, 10088, 9768, 9761]
# Sum 429503
# Mean 9988.441860465116
# Min 9753
# Max 10250

print("Array", sample_list)
print("Sum", np.sum(np.asarray(sample_list)))
print("Mean", np.mean(np.asarray(sample_list)))
print("Min", np.min(np.asarray(sample_list)))
print("Max", np.max(np.asarray(sample_list)))

Array [9944, 10056, 9867, 9818, 9794, 10123, 10021, 10108, 9795, 10000, 9965, 10063, 9860, 10211, 9941, 9948, 10099, 9827, 10088, 10185, 9923, 10234, 10020, 10146, 9990, 9956, 9887, 9753, 9854, 10227, 9791, 10015, 10060, 10250, 9933, 9860, 9832, 10155, 10054, 10233, 10088, 9768, 9761]
Sum 429503
Mean 9988.441860465116
Min 9753
Max 10250


In [5]:
# Applying the augmentation for images in the based on the random number generated in the range of 0 to 4.

def Augment_images(img, choice):
    
    if choice == 0:
        PerspectiveTransform = A.Compose([A.Perspective(scale=(0.05, 0.1), keep_size=True, pad_mode=0, pad_val=0, 
                                    mask_pad_val=0, fit_output=False, interpolation=1, always_apply=True, p=1)])
        transformedimg = PerspectiveTransform(image=img)['image']
        
    elif choice ==1:
        ScaleRotation = A.Compose([A.ShiftScaleRotate(shift_limit=0.075, scale_limit=0.1, rotate_limit=60, 
                                    interpolation=1, border_mode=4, value=None, mask_value=None, 
                                    shift_limit_x=None, shift_limit_y=None, always_apply=True, p=1)])
            
        transformedimg = ScaleRotation(image=img)['image']
    
    elif choice == 2:
        MotionBlur = A.Compose([A.MotionBlur(blur_limit=(3, 7), always_apply=True, p=1)])
        
        transformedimg = MotionBlur(image=img)['image']
        
    elif choice == 3:
        randomRotate = A.Compose([A.Rotate(limit=[-50, 50], interpolation=cv2.INTER_LINEAR, border_mode=4, 
                                    value=None, mask_value=None, always_apply=True, p=1.0)])
        transformedimg = randomRotate(image=img)['image']
        
    else:
        flipHorizontal = A.Compose([A.HorizontalFlip])
        transformedimg = flipHorizontal(image=img)['image']
            
    return transformedimg

In [6]:
# The classes in the list if flipped will contridict the pattern of another sign belonging to different class.
NO_HFLIP = [0, 1, 2, 3, 4, 5, 6, 7, 8, 14, 19, 20, 33, 34, 36, 37, 38, 39]

for dirname in os.listdir('Training_Data'):
    
    auged_samples = sample_list[int(dirname)] - len(os.listdir(os.path.join('Training_Data', dirname)))
    
    samples = os.listdir(os.path.join('Training_Data', dirname))
    
    # collecting the seed value from the seed list created to ensure that same random number is generated in every execution.
    random.seed(seed_list[int(dirname)])
    
    # Augmenting the original images until the set number of images per class is reached.
    while auged_samples >= 0:
        for sample in samples:
            try:
                #load the image
                img = cv2.imread(os.path.join('Training_Data', dirname, sample))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                # Apply the transformation
                if int(dirname) not in NO_HFLIP:
                    img = Augment_images(img, random.randint(0, 4))
                else:
                    img = Augment_images(img, random.randint(0, 3))
                # the augmented image.
                name, extn = os.path.splitext(sample)
                fname = f'aug_{auged_samples}_{name}.{extn}'
                auged_samples = auged_samples - 1
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                cv2.imwrite(os.path.join('Training_Data', dirname, fname), img)
            
            except Exception as e:
                print(e)
            
            if auged_samples < 0:
                break